In [1]:
import cv2
import numpy as np
import face_recognition
import pandas as pd
import os
import time
from datetime import datetime
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from collections import Counter

In [2]:
current_time = datetime.now().time()
start_time = datetime.strptime("08:00:00", "%H:%M:%S").time()
end_time = datetime.strptime("14:00:00", "%H:%M:%S").time()

if not (start_time <= current_time <= end_time):
    print("Current TIme: ", current_time)
    print("Application can only run between 8 AM and 9 AM.")
    exit()

In [3]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
path = 'Training_Images/Training_Images'
images = []
classNames = []
myList = os.listdir(path)
print("Training images:", myList)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

Training images: ['Chetana.jpg', 'Jayadas.jpg', 'Mrunmayee.jpg', 'Netra.jpg', 'Yash.jpg']


In [4]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(img)
        if len(encodings) > 0:
            encodeList.append(encodings[0])
    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding Complete')

Encoding Complete


In [ ]:
# def markAttendance(name):
#     with open('Attendance.csv', 'r+') as f:
#         myDataList = f.readlines()


#         nameList = []
#         for line in myDataList:
#             entry = line.split(',')
#             nameList.append(entry[0])
#             if name not in nameList:
#                 now = datetime.now()
#                 dtString = now.strftime("%m/%d/%Y,%H:%M:%S")
#                 f.writelines(f'\n{name},{dtString}')

In [ ]:
#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr

In [5]:
emotion_model = load_model("emotion_model.h5")

In [6]:
emotion_labels = ['Angry', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [7]:
csv_file = "Attendance.csv"
if not os.path.exists(csv_file) or os.stat(csv_file).st_size == 0:
    df = pd.DataFrame(columns = ["Name", "Date", "Time", "Emotion"])
    df.to_csv(csv_file, index = False)

In [8]:
def markAttendance(name, emotion):
    now = datetime.now()
    date_today = now.strftime("%Y-%m-%d")
    time_now = now.strftime("%H:%M:%S")

    new_entry = pd.DataFrame([[name, date_today, time_now, emotion]], 
                              columns = ["Name", "Date", "Time", "Emotion"])

    new_entry.to_csv(csv_file, mode = 'a', header=not os.path.exists(csv_file), index = False)
    
    print(f"Attendance Marked for {name} at {time_now} with Emotion: {emotion}")

In [10]:
cap = cv2.VideoCapture(0)
start_time = time.time()  
emotion_counts = Counter()  

while time.time() - start_time < 5: 
    success, img = cap.read()
    if not success:
        break

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        if encodeFace is None or len(encodeFace) == 0:
            continue  

        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        if not matches:
            continue  

        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        if len(faceDis) == 0:
            continue

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            face_roi = img[y1:y2, x1:x2]
            if face_roi.size == 0:
                print("Empty face region detected, skipping")
                continue

            face_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
            face_gray = cv2.resize(face_gray, (48, 48))
            face_gray = face_gray.astype("float32") / 255.0  
            face_gray = np.expand_dims(face_gray, axis = -1) 
            face_gray = np.expand_dims(face_gray, axis = 0)   

            emotion_prediction = emotion_model.predict(face_gray)
            emotion_label = emotion_labels[np.argmax(emotion_prediction)]

            emotion_counts[emotion_label] += 1

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f"{name} ({emotion_label})", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Webcam", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if emotion_counts:
    most_common_emotion = emotion_counts.most_common(1)[0][0]  
    markAttendance(name, most_common_emotion)  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Attendance Marked for CHETANA at 13:27:19 with Emotion: Happy
